In [ ]:
import pandas as pd
import numpy as np 
import logging
from pathlib import Path
import sys

In [2]:
""" 
Script para crear la dimensión de calificaciones sanitarias de NYC.
"""


# --- Configuración de Logging Profesional ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s.%(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
log = logging.getLogger(__name__)

# --- Definición de Rutas ---
try:
    # *** Revisa y ajusta RUTA_BASE ***
    RUTA_BASE = Path(r"H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS").resolve()
    RUTA_PROCESSED = RUTA_BASE / "data" / "processed"
    RUTA_DIM = RUTA_PROCESSED / "dim"

    # Ruta para la NUEVA dimensión de calificaciones sanitarias
    RUTA_SALIDA_DIM_CALIFICACION = (RUTA_DIM / "dim_calificacion_sanitaria.csv").resolve()

    # Crear directorio si no existe
    RUTA_DIM.mkdir(parents=True, exist_ok=True)

except Exception as e:
    log.exception(f"Error crítico al definir rutas: {e}")
    sys.exit(1)

# --- Funciones ---

def crear_y_guardar_dimension_calificacion(ruta_salida: Path) -> bool:
    """
    Crea y guarda la tabla de dimensión para las calificaciones sanitarias de NYC.
    La información se define manualmente basada en las reglas del DOHMH.

    Args:
        ruta_salida (Path): Ruta donde guardar el archivo CSV de la dimensión.

    Returns:
        bool: True si se guardó correctamente, False en caso contrario.
    """
    log.info("Definiendo datos para la dimensión de calificaciones sanitarias...")

    # Definimos manualmente los datos de la dimensión según las reglas conocidas
    # Usamos np.inf para el límite superior de 'C'. Usamos pd.NA para puntajes no aplicables.
    datos_calificaciones = [
        {'calificacion': 'A', 'descripcion': 'Excelente', 'puntaje_min': 0, 'puntaje_max': 13},
        {'calificacion': 'B', 'descripcion': 'Bueno', 'puntaje_min': 14, 'puntaje_max': 27},
        {'calificacion': 'C', 'descripcion': 'Satisfactorio / Requiere Mejora', 'puntaje_min': 28, 'puntaje_max': np.inf}, # Infinito o un número muy alto
        {'calificacion': 'P', 'descripcion': 'Calificación Pendiente', 'puntaje_min': pd.NA, 'puntaje_max': pd.NA}, # Grade Pending (usualmente después de B/C inicial)
        {'calificacion': 'Z', 'descripcion': 'Calificación Pendiente', 'puntaje_min': pd.NA, 'puntaje_max': pd.NA}, # Grade Pending (a veces usado)
        {'calificacion': 'N', 'descripcion': 'Aún No Calificado', 'puntaje_min': pd.NA, 'puntaje_max': pd.NA}, # Not Yet Graded (nuevo establecimiento)
        # Podríamos añadir una fila para representar calificaciones nulas o desconocidas si aparecen en los datos
        {'calificacion': 'Desconocido', 'descripcion': 'Calificación Desconocida o No Aplicable', 'puntaje_min': pd.NA, 'puntaje_max': pd.NA}
    ]

    try:
        # Crear DataFrame
        dim_calificacion = pd.DataFrame(datos_calificaciones)

        # Asegurar tipos de datos
        # Usamos category para la calificación, string para descripción, Int64 (nullable) para puntajes
        dim_calificacion['calificacion'] = dim_calificacion['calificacion'].astype('category')
        dim_calificacion['descripcion'] = dim_calificacion['descripcion'].astype('string')
        # Convertir puntajes a numérico (float para permitir np.inf) y luego a Int64 si no hay inf
        dim_calificacion['puntaje_min'] = pd.to_numeric(dim_calificacion['puntaje_min'], errors='coerce')
        dim_calificacion['puntaje_max'] = pd.to_numeric(dim_calificacion['puntaje_max'], errors='coerce')

        # Intentar convertir a Int64 si no hay infinitos (para mejor manejo de nulos que float)
        # Nota: np.inf no puede estar en Int64. Lo dejaremos como float por ahora debido al np.inf.
        # dim_calificacion['puntaje_min'] = dim_calificacion['puntaje_min'].astype('Int64')
        # dim_calificacion['puntaje_max'] = dim_calificacion['puntaje_max'].astype('Int64') # Esto fallaría por np.inf

        log.info(f"Dimensión de calificaciones creada con {len(dim_calificacion)} registros.")
        log.debug(f"Tipos de datos finales:\n{dim_calificacion.dtypes}")

        # Guardar la dimensión
        log.info(f"Guardando dimensión de calificaciones en: {ruta_salida}")
        dim_calificacion.to_csv(ruta_salida, index=False, encoding='utf-8-sig')
        log.info("Dimensión de calificaciones guardada exitosamente.")
        return True

    except Exception as e:
        log.exception(f"Error creando o guardando la dimensión de calificaciones: {e}")
        return False

# --- Flujo Principal ---
def main_crear_dim_calificacion():
    """Orquesta la creación y guardado de la dimensión de calificaciones."""
    log.info("================================================")
    log.info("=== INICIO: Crear Dimensión Calificación Sanitaria ===")
    log.info("================================================")

    exito = crear_y_guardar_dimension_calificacion(RUTA_SALIDA_DIM_CALIFICACION)

    log.info("================================================")
    if exito:
        log.info("=== Proceso completado exitosamente ===")
        log.info(f"Archivo creado en: {RUTA_SALIDA_DIM_CALIFICACION}")
    else:
        log.error("=== Proceso falló al crear/guardar la dimensión ===")
    log.info("================================================")

if __name__ == "__main__":
    main_crear_dim_calificacion()

2025-04-19 13:11:55 [INFO] __main__.main_crear_dim_calificacion: ================================================
2025-04-19 13:11:55 [INFO] __main__.main_crear_dim_calificacion: === INICIO: Crear Dimensión Calificación Sanitaria ===
2025-04-19 13:11:55 [INFO] __main__.main_crear_dim_calificacion: ================================================
2025-04-19 13:11:55 [INFO] __main__.crear_y_guardar_dimension_calificacion: Definiendo datos para la dimensión de calificaciones sanitarias...
2025-04-19 13:11:55 [INFO] __main__.crear_y_guardar_dimension_calificacion: Dimensión de calificaciones creada con 7 registros.
2025-04-19 13:11:55 [INFO] __main__.crear_y_guardar_dimension_calificacion: Guardando dimensión de calificaciones en: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_calificacion_sanitaria.csv
2025-04-19 13:11:55 [INFO] __main__.crear_y_guardar_dimension_calificacion: Dimensión de calificaciones guardada exitosamente.
2025-04-19 13:11:55 [INFO] __main__.ma